<a href="https://colab.research.google.com/github/sukantamitra007/GenAILearning/blob/TransformerModel/Project_Work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Prepare Dataset for Fine Tunning
1. Remove all review which having review comment as 2 to remove the baise

Models for text summarization
  summarization in few-shot settings T5

 Evaltution metrics is the ROUGE score (short for Recall-Oriented Understudy for Gisting Evaluation).

In [1]:
pip install gradio

In [2]:
import gradio as gr
def greet(name):
    return "Hello " + name
demo = gr.Interface(fn=greet, inputs="text", outputs="text")
demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://22492c87284250bb92.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


# Lets Install Necessary libraries

In [1]:
pip install transformers[torch]  datasets evaluate rouge_score

# Load BillSum dataset

In [2]:
from datasets import load_dataset

billsum = load_dataset("billsum", split="ca_test")
billsum = billsum.train_test_split(test_size=0.99)
billsum_train= billsum['train']   # Doing this to reduce the data volumn
billsum = billsum_train.train_test_split(test_size=0.40)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
billsum

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 7
    })
    test: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 5
    })
})

# Preprocess

In [12]:
from transformers import AutoTokenizer

checkpoint = "google-t5/t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# preprocessing function

In [13]:
prefix = "summarize: "

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]  # Add prefix  as required for the model
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True) # add truncation on and max length

    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True) # add target truncation on

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [14]:
tokenized_billsum = billsum.map(preprocess_function, batched=True) #setting batched=True to process multiple elements of the dataset at

Map:   0%|          | 0/7 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

In [15]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint) #dynamically pad the sentences to the longest length in a batch during collation.

# Evaluate

In [16]:
import evaluate
rouge = evaluate.load("rouge")

In [17]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

# Train

In [18]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [19]:
training_args = Seq2SeqTrainingArguments(
    output_dir="my_project",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=4,
    predict_with_generate=True,
    fp16=False#,
    #push_to_hub=True,
)

In [20]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_billsum["train"],
    eval_dataset=tokenized_billsum["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,4.549047,0.213800,0.113300,0.179700,0.179900,19.000000
2,No log,4.502214,0.213800,0.113300,0.179700,0.179900,19.000000
3,No log,4.470104,0.213800,0.113300,0.179700,0.179900,19.000000
4,No log,4.454156,0.213800,0.113300,0.179700,0.179900,19.000000


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


TrainOutput(global_step=4, training_loss=4.661494731903076, metrics={'train_runtime': 307.3041, 'train_samples_per_second': 0.091, 'train_steps_per_second': 0.013, 'total_flos': 7579140882432.0, 'train_loss': 4.661494731903076, 'epoch': 4.0})

In [26]:
 trainer.save_model("SummarizationFineTunedModel")

In [21]:
text = "summarize: The Inflation Reduction Act lowers prescription drug costs, health care costs, and energy costs. It's the most aggressive action on tackling the climate crisis in American history, which will lift up American workers and create good-paying, union jobs across the country. It'll lower the deficit and ask the ultra-wealthy and corporations to pay their fair share. And no one making under $400,000 per year will pay a penny more in taxes."

In [28]:
from transformers import pipeline

summarizer = pipeline("summarization", model="SummarizationFineTunedModel")
summarizer(text,max_length=51)



[{'summary_text': 'the Inflation Reduction Act lowers prescription drug costs, health care costs, and energy costs . it will ask the ultra-wealthy and corporations to pay their fair share . no one making under $400,000 per year will'}]